<a href="https://colab.research.google.com/github/adolfolh/casuality-classification/blob/main/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run to download data
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/casualties-2020.csv
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/accidents-2020.csv